In [1]:
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from mpl_toolkits.mplot3d import Axes3D
import gymnasium
from run.train import configuration
import json
from copy import copy, deepcopy
import panda_gym
import pybullet
from sb3_contrib import TQC
import winsound

C:\Users\eclip\GitHub\panda-gym\venv\Lib\site-packages\gymnasium-0.28.1-py3.11.egg\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment PandaReachAO-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
# standard settings
reward_type="kumar_her"
n_substeps=20
max_ep_steps = 200
goal_condition="reach"
#register envs
panda_gym.register_reach_ao(max_ep_steps)

# visual stuff
sphere_list = []
sphere_count = 0

In [3]:
# training env
evaluation_scenario="wangexp_3"
prior_orientation="fkine"
show_goal_space = False
cameraTargetPosition = (0.06289377063512802, 0.05091303586959839, 0.27599984407424927)
cameraDistance = 1.359999656677246
cameraPitch = -33.20000076293945
cameraYaw = 51.800025939941406

# narrow_tunnel
evaluation_scenario="narrow_tunnel"
prior_orientation="left"
show_goal_space = True
cameraTargetPosition = (0.6126745939254761, -0.002699121832847595, 0.3287460505962372)
cameraDistance = 1.119999885559082
cameraPitch = -42.800045013427734
cameraYaw = 221.39991760253906

# library 2
evaluation_scenario="library2"
prior_orientation="back"

# workshop
evaluation_scenario="workshop"
prior_orientation="fkine"

# wall
evaluation_scenario="wall"
prior_orientation="fkine"

In [4]:
# get env
work_env = gymnasium.make(configuration["env_name"], render=False, control_type="js",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition=goal_condition,
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=False, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     joint_obstacle_observation="vectors+all",
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=False, n_substeps=n_substeps)


In [5]:
# get rendered env
env = gymnasium.make(configuration["env_name"],
                     render=True, control_type="js",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition=goal_condition,
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=show_goal_space, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     joint_obstacle_observation="vectors+all",
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=False, n_substeps=n_substeps)

pybullet.resetDebugVisualizerCamera(cameraDistance=cameraDistance, cameraYaw=cameraYaw, cameraPitch=cameraPitch, cameraTargetPosition=cameraTargetPosition, physicsClientId=1)



work_env.robot.control_type = "jsd"
results, metrics = evaluate.evaluate_ensemble(["prior"], work_env, human=human, num_episodes=100, deterministic=True,
                                     strategy="",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation=prior_orientation)
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]

In [6]:
num_episodes = 50
work_env.robot.control_type = "js"
model_names = ["gallant-serenity-299", "deep-frog-298", "solar-microwave-297", "revived-serenity-296",
                "glamorous-resonance-295"]
ensemble = []
for model_name in model_names:
    ensemble.append(TQC.load(fr"../run/run_data/wandb/{model_name}/files/best_model.zip", env=work_env,
                     custom_objects={"action_space": gymnasium.spaces.Box(-1.0, 1.0, shape=(7,),dtype=np.float32)}))  # for some reason it won't read action space sometimes)

results, metrics = evaluate.evaluate_ensemble(ensemble, work_env, human=False, num_episodes=num_episodes, deterministic=True,
                                     strategy="variance_only",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation="")
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]
work_env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


wangexp_3: 100%|██████████| 50/50 [00:28<00:00,  1.75it/s]


pybullet.setRealTimeSimulation(0)
loggingID = pybullet.startStateLogging(loggingType=pybullet.STATE_LOGGING_VIDEO_MP4, fileName = f"videos/{evaluation_scenario}_mt-cl-ensemble.mp4", physicsClientId=1)

In [7]:
env.robot.control_type = "js"
#pybullet.setRealTimeSimulation(enableRealTimeSimulation=1, physicsClientId=1)

for model_name in model_names:
    ensemble.append(TQC.load(fr"../run/run_data/wandb/{model_name}/files/best_model.zip", env=env,
                     custom_objects={"action_space": gymnasium.spaces.Box(-1.0, 1.0, shape=(7,),dtype=np.float32)}))  # for some reason it won't read action space sometimes)

_, _ = evaluate.evaluate_ensemble(ensemble, env, human=True, num_episodes=num_episodes, deterministic=True,
                                     strategy="variance_only",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation="",
                                  pre_calc_metrics=metrics)


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


wangexp_3: 100%|██████████| 50/50 [00:45<00:00,  1.09it/s]


pybullet.stopStateLogging(loggingID)
env.close()

In [8]:
env.close()

work_env.robot.control_type = "js"
model_name = "firm-pond-79"
model = TQC.load(fr"../run/run_data/wandb/{model_name}/files/best_model.zip", env=work_env,
                 custom_objects={"action_space": gymnasium.spaces.Box(-1.0, 1.0, shape=(7,),
                                                                      dtype=np.float32)})  # for some reason it won't read action space sometimes

results, metrics = evaluate.evaluate_ensemble([model], work_env, human=human, num_episodes=100, deterministic=True,
                                     strategy="variance_only",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation="")
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]

# get rendered env
env = gymnasium.make(configuration["env_name"],
                     render=True, control_type="js",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition="halt",
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=False, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     joint_obstacle_observation="vectors+all",
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=True, n_substeps=n_substeps)

In [9]:

# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=1, rgbaColor=[1,1,1,0.5], physicsClientId=1)
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=3, rgbaColor=[1,1,1,1], physicsClientId=1)
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=2, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=1, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=0, rgbaColor=[1,1,1,0.5])

In [10]:
#env.close()